<a href="https://colab.research.google.com/github/algo21-221040114/PytorchTest/blob/main/Model_Transformer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install sweetviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 7.0 MB/s eta 0:00:00


In [5]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     | 22.6 MB 20.7 MB/s 0:00:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 6.4 MB/s eta 0:00:00
  Created wheel for ydata-profiling: filename=ydata_profiling-0.0.dev0-py2.py3-none-any.whl size=345010 sha256=f5e3443945adc0ec14c769669df0b0ea323a8d22ad7d4f454facd44d32623476
  Stored in directory: /tmp/pip-ephem-wheel-cache-vda36wy_/wheels/43/c8/f4/c0ebc32d7f20fe89d0e92d90eaeef5f0c0594a89b6bc16b352
Successfully built ydata-profiling
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.

In [6]:
import pandas as pd
import numpy as np
import sweetviz as sv
from pandas_profiling import ProfileReport

import math
import torch
import torch.nn as nn

import seaborn as sns
import matplotlib.pyplot as plt

<ipython-input-6-609273d2979d>:4: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [9]:
df = pd.read_csv('/content/sample_data/000001.SS.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-16,3107.669922,3177.790039,3023.300049,3170.709961,3170.709961,444700
2022-03-17,3215.010010,3260.169922,3202.929932,3215.040039,3215.040039,420800
2022-03-18,3207.149902,3260.780029,3197.360107,3251.070068,3251.070068,373000
2022-03-21,3255.620117,3267.500000,3223.389893,3253.689941,3253.689941,354200
2022-03-22,3249.540039,3279.110107,3239.530029,3259.860107,3259.860107,361200


In [ ]:
# Plot
fig, ax = plt.subplots()
ax.plot(df.index, df['High'], color='y', label='high')
ax.plot(df.index, df['Close'], color='b', label='close')
ax.set_title('000001.SS')
ax.legend()
plt.show()

In [ ]:
# EDA 
def eda_sweetviz(df, report_name, target=None):
    # target certain data
    sweet_report = sv.analyze(df, target_feat=target)
    # in notebook, download the file and open in the browser
    sweet_report.show_html(f'{report_name}.html')

def eda_sweetviz_compare(df1, df2, report_name, feature, target):
    # compare two datasets
    feature_config = sv.FeatureConfig(force_text=feature, force_cat=feature)
    sweet_report = sv.compare(df1, df2, feat_cfg=feature_config, target_feat=target)
    sweet_report.show_html(f'{report_name}_compare.html')


eda_sweetviz(df, 'Daily Data of 000001.SS')

def eda_pandas_profilling(df, report_name):
    # directly show in notebook
    profile = ProfileReport(df, title=report_name, explorative=True)
    return profile

eda_pandas_profilling(df, '000001.SS Report')

In [10]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)*(-math.log(10000.0)/d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [50]:
# 策略：使用前十个交易日的close信息预测当日close
class QuantModel(nn.Module):

  def __init__(self):
      super(QuantModel, self).__init__()

      self.embedding = nn.Embedding(num_embeddings=10, embedding_dim=64)
      self.positional_encoding = PositionalEncoding(d_model=64, dropout=0.2)
      self.transformer = nn.Transformer(d_model=64,
                                        nhead=2,
                                        num_encoder_layers=12,
                                        num_decoder_layers=12,
                                        dim_feedforward=512,
                                        dropout=0.2,
                                        activation='relu',
                                        custom_encoder=None,
                                        custom_decoder=None,batch_first=True)
      self.out_linear = nn.Linear(64, 10)

  def forward(self, src, tgt):

      tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size()[-1])

      src = self.embedding(src)
      tgt = self.embedding(tgt)
        
      src = self.positional_encoding(src)
      tgt = self.positional_encoding(tgt)
      
      out = self.transformer(src, tgt, tgt_mask=tgt_mask)

      return out


In [51]:
model = QuantModel()

In [52]:
batch_size = 5
src = torch.LongTensor([[0, 3, 4, 5, 6, 1, 2, 2]])
tgt = torch.LongTensor([[3, 4, 5, 6, 1, 2, 2]])
out = model(src, tgt)
print(out.size())

torch.Size([1, 8])
torch.Size([1, 7])
torch.Size([1, 7, 64])


In [ ]:
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
total_loss = 0

for step in range(1000):

    src = 
    tgt =  

    optimizer.zero_grad()
    out = model(src, tgt)
    out = model.out_linear(out)

    loss = 

    loss.backward()
    optimizer.step()
    total_loss += loss

    # 每40次打印一下loss
    if step != 0 and step % 40 == 0:
        print("Step {}, total_loss: {}".format(step, total_loss))
        total_loss = 0